Exploring all runnable functionalities langchain

In [24]:
from langchain.schema.runnable import(
    RunnableBranch,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough
)
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

Runnable pass through maps the previous values to the next as it is usually used for branching


In [ ]:
# Pass through examples
skip = RunnableLambda(lambda x: x)
skip_alt  = RunnablePassthrough
# Runnable lambda to convert functions into chainnable components
def add_preface(x, preface = ""):
    if type(x) == str:
        x = preface + x
    print(x)
    return x
def Rpreface(preface=""):
    return RunnableLambda(partial(add_preface, preface=preface))

chain = skip| Rpreface
chain.invoke()
